# Bengali.AI SEResNeXt training with pytorch

I will introduce following contents

 - **Fast data loading** with feather format
 - **Data augmentation** technic with affine transformation
 - **CNN SoTA models**: Use pytorch `pretrainedmodels` library, especially I use **`SEResNeXt`** in this notebook
 - **Training code abstraction**: Use `pytorch-ignite` module for the trainining abstraction
 
### Update history

 - 2020/1/4 v2: Added albumentations augmentations introduced in [Bengali: albumentations data augmentation tutorial](https://www.kaggle.com/corochann/bengali-albumentations-data-augmentation-tutorial)

# Table of Contents:
**[Fast data loading with feather](#load)**<br>
**[Dataset](#dataset)**<br>
**[Data augmentation/processing](#processing)**<br>
**[pytorch model & define classifier](#model)**<br>
**[Training code](#train)**<br>
**[Prediction](#pred)**<br>
**[Reference and further reading](#ref)**<br>

To install https://github.com/Cadene/pretrained-models.pytorch without internet connection, we can install library as "dataset".

It is uploaded by @rishabhiitbhu : https://www.kaggle.com/rishabhiitbhu/pretrainedmodels

!pip install iterative-stratification

In [1]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

In [2]:
import pretrainedmodels

In [3]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [4]:
import gc
import os
from pathlib import Path
import random
import sys

from tqdm.notebook import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

# --- plotly ---
#from plotly import tools, subplots
#import plotly.offline as py
#py.init_notebook_mode(connected=True)
#import plotly.graph_objs as go
#import plotly.express as px
#import plotly.figure_factory as ff

# --- models ---
#from sklearn import preprocessing
#from sklearn.model_selection import KFold
#import lightgbm as lgb
#import xgboost as xgb
#import catboost as cb

# --- setup ---
pd.set_option('max_columns', 50)

import warnings
warnings.filterwarnings('ignore')

In [5]:
import torch 
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
seed_everything(SEED)

In [6]:
debug=False
submission=False
batch_size = 256
device='cuda:0'
out='.'
image_size=64
arch='pretrained'
model_name='se_resnext50_32x4d'

In [7]:
datadir = Path('../bengali/input/')
featherdir = Path('../bengali/feather/')
outdir = Path('../bengali/model/64_base_Cutout_v1_rotate_v2')

In [8]:
# Read in the data CSV files
# train = pd.read_csv(datadir/'train.csv')
# test = pd.read_csv(datadir/'test.csv')
# sample_submission = pd.read_csv(datadir/'sample_submission.csv')
# class_map = pd.read_csv(datadir/'class_map.csv')

# Fast data loading with feather

Refer [Bengali.AI super fast data loading with feather](https://www.kaggle.com/corochann/bengali-ai-super-fast-data-loading-with-feather) and [dataset](https://www.kaggle.com/corochann/bengaliaicv19feather) for detail.<br/>
Original `parquet` format takes about 60 sec to load 1 data, while `feather` format takes about **2 sec to load 1 data!!!**

### How to add dataset

When you write kernel, click "+ Add Data" botton on right top.<br/>
Then inside window pop-up, you can see "Search Datasets" text box on right top.<br/>
You can type "bengaliai-cv19-feather" to find this dataset and press "Add" botton to add the data.

In [9]:
import numpy as np
import pandas as pd
import gc


def prepare_image(datadir, featherdir, data_type='train',
                  submission=False, indices=[0, 1, 2, 3]):
    assert data_type in ['train', 'test']
    if submission:
        image_df_list = [pd.read_parquet(datadir / f'{data_type}_image_data_{i}.parquet')
                         for i in indices]
    else:
        image_df_list = [pd.read_feather(featherdir / f'{data_type}_image_data_{i}.feather')
                         for i in indices]

    print('image_df_list', len(image_df_list))
    HEIGHT = 137
    WIDTH = 236
    images = [df.iloc[:, 1:].values.reshape(-1, HEIGHT, WIDTH) for df in image_df_list]
    del image_df_list
    gc.collect()
    images = np.concatenate(images, axis=0)
    return images


In [10]:
%%time

train = pd.read_csv(datadir/'train.csv')
train_labels = train[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values
indices = [0] if debug else [0, 1, 2, 3]
train_images = prepare_image(
    datadir, featherdir, data_type='train', submission=False, indices=indices)


image_df_list 4
CPU times: user 14.1 s, sys: 16.2 s, total: 30.3 s
Wall time: 4.86 s


<a id="dataset"></a>
# Dataset

In [11]:
"""
Referenced `chainer.dataset.DatasetMixin` to work with pytorch Dataset.
"""
import numpy
import six
import torch
from torch.utils.data.dataset import Dataset


class DatasetMixin(Dataset):

    def __init__(self, transform=None):
        self.transform = transform

    def __getitem__(self, index):
        """Returns an example or a sequence of examples."""
        if torch.is_tensor(index):
            index = index.tolist()
        if isinstance(index, slice):
            current, stop, step = index.indices(len(self))
            return [self.get_example_wrapper(i) for i in
                    six.moves.range(current, stop, step)]
        elif isinstance(index, list) or isinstance(index, numpy.ndarray):
            return [self.get_example_wrapper(i) for i in index]
        else:
            return self.get_example_wrapper(index)

    def __len__(self):
        """Returns the number of data points."""
        raise NotImplementedError

    def get_example_wrapper(self, i):
        """Wrapper of `get_example`, to apply `transform` if necessary"""
        example = self.get_example(i)
        if self.transform:
            example = self.transform(example)
        return example

    def get_example(self, i):
        """Returns the i-th example.

        Implementations should override it. It should raise :class:`IndexError`
        if the index is invalid.

        Args:
            i (int): The index of the example.

        Returns:
            The i-th example.

        """
        raise NotImplementedError


This `DatasetMixin` class can be used to define any custom dataset class in pytorch. We can implement `get_example(self, i)` method to return `i`-th data.

Here I return i-th image `x` and `label`, with scaling image to be value ranges between 0~1.

In [12]:
import numpy as np


class BengaliAIDataset(DatasetMixin):
    def __init__(self, images, labels=None, transform=None, indices=None):
        super(BengaliAIDataset, self).__init__(transform=transform)
        self.images = images
        self.labels = labels
        if indices is None:
            indices = np.arange(len(images))
        self.indices = indices
        self.train = labels is not None

    def __len__(self):
        """return length of this dataset"""
        return len(self.indices)

    def get_example(self, i):
        """Return i-th data"""
        i = self.indices[i]
        x = self.images[i]
        # Opposite white and black: background will be white and
        # for future Affine transformation
        x = (255 - x).astype(np.float32) / 255.
        if self.train:
            y = self.labels[i]
            return x, y
        else:
            return x


Let's see how this `BengaliAIDataset` work

`train_dataset[i]` returns i-th image array and 3 target labels (graphme_root, vowel_diacritic and consonant_diacritic).

<a id="processing"></a>
# Data augmentation/processing

For CNN training, data augmentation is important to improve test accuracy (generalization performance). I will show some image preprocessing to increase the data variety.

In [13]:
"""
From https://www.kaggle.com/corochann/deep-learning-cnn-with-chainer-lb-0-99700
"""
import cv2
from skimage.transform import AffineTransform, warp
import numpy as np


def affine_image(img):
    """

    Args:
        img: (h, w) or (1, h, w)

    Returns:
        img: (h, w)
    """
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return transformed_image


def crop_char_image(image, threshold=5./255.):
    assert image.ndim == 2
    is_black = image > threshold

    is_black_vertical = np.sum(is_black, axis=0) > 0
    is_black_horizontal = np.sum(is_black, axis=1) > 0
    left = np.argmax(is_black_horizontal)
    right = np.argmax(is_black_horizontal[::-1])
    top = np.argmax(is_black_vertical)
    bottom = np.argmax(is_black_vertical[::-1])
    height, width = image.shape
    cropped_image = image[left:height - right, top:width - bottom]
    return cropped_image


def resize(image, size=(128, 128)):
    return cv2.resize(image, size)

Put everything together with `Transform` class. <br>
[Update] I added **albumentations augmentations** introduced in [Bengali: albumentations data augmentation tutorial](https://www.kaggle.com/corochann/bengali-albumentations-data-augmentation-tutorial).

In [14]:
import albumentations
from albumentations.core.transforms_interface import DualTransform
from albumentations.augmentations import functional as F
import albumentations as A
import numpy as np

class GridMask(DualTransform):
    """GridMask augmentation for image classification and object detection.
    
    Author: Qishen Ha
    Email: haqishen@gmail.com
    2020/01/29

    Args:
        num_grid (int): number of grid in a row or column.
        fill_value (int, float, lisf of int, list of float): value for dropped pixels.
        rotate ((int, int) or int): range from which a random angle is picked. If rotate is a single int
            an angle is picked from (-rotate, rotate). Default: (-90, 90)
        mode (int):
            0 - cropout a quarter of the square of each grid (left top)
            1 - reserve a quarter of the square of each grid (left top)
            2 - cropout 2 quarter of the square of each grid (left top & right bottom)

    Targets:
        image, mask

    Image types:
        uint8, float32

    Reference:
    |  https://arxiv.org/abs/2001.04086
    |  https://github.com/akuxcw/GridMask
    """

    def __init__(self, num_grid=3, fill_value=0, rotate=0, mode=0, always_apply=False, p=0.5):
        super(GridMask, self).__init__(always_apply, p)
        if isinstance(num_grid, int):
            num_grid = (num_grid, num_grid)
        if isinstance(rotate, int):
            rotate = (-rotate, rotate)
        self.num_grid = num_grid
        self.fill_value = fill_value
        self.rotate = rotate
        self.mode = mode
        self.masks = None
        self.rand_h_max = []
        self.rand_w_max = []

    def init_masks(self, height, width):
        if self.masks is None:
            self.masks = []
            n_masks = self.num_grid[1] - self.num_grid[0] + 1
            for n, n_g in enumerate(range(self.num_grid[0], self.num_grid[1] + 1, 1)):
                grid_h = height / n_g
                grid_w = width / n_g
                this_mask = np.ones((int((n_g + 1) * grid_h), int((n_g + 1) * grid_w))).astype(np.uint8)
                for i in range(n_g + 1):
                    for j in range(n_g + 1):
                        this_mask[
                             int(i * grid_h) : int(i * grid_h + grid_h / 2),
                             int(j * grid_w) : int(j * grid_w + grid_w / 2)
                        ] = self.fill_value
                        if self.mode == 2:
                            this_mask[
                                 int(i * grid_h + grid_h / 2) : int(i * grid_h + grid_h),
                                 int(j * grid_w + grid_w / 2) : int(j * grid_w + grid_w)
                            ] = self.fill_value
                
                if self.mode == 1:
                    this_mask = 1 - this_mask

                self.masks.append(this_mask)
                self.rand_h_max.append(grid_h)
                self.rand_w_max.append(grid_w)

    def apply(self, image, mask, rand_h, rand_w, angle, **params):
        h, w = image.shape[:2]
        mask = F.rotate(mask, angle) if self.rotate[1] > 0 else mask
        mask = mask[:,:,np.newaxis] if image.ndim == 3 else mask
        image *= mask[rand_h:rand_h+h, rand_w:rand_w+w].astype(image.dtype)
        return image

    def get_params_dependent_on_targets(self, params):
        img = params['image']
        height, width = img.shape[:2]
        self.init_masks(height, width)

        mid = np.random.randint(len(self.masks))
        mask = self.masks[mid]
        rand_h = np.random.randint(self.rand_h_max[mid])
        rand_w = np.random.randint(self.rand_w_max[mid])
        angle = np.random.randint(self.rotate[0], self.rotate[1]) if self.rotate[1] > 0 else 0

        return {'mask': mask, 'rand_h': rand_h, 'rand_w': rand_w, 'angle': angle}

    @property
    def targets_as_params(self):
        return ['image']

    def get_transform_init_args_names(self):
        return ('num_grid', 'fill_value', 'rotate', 'mode')

def add_gaussian_noise(x, sigma):
    x += np.random.randn(*x.shape) * sigma
    x = np.clip(x, 0., 1.)
    return x


def _evaluate_ratio(ratio):
    if ratio <= 0.:
        return False
    return np.random.uniform() < ratio


def apply_aug(aug, image):
    return aug(image=image)['image']


class Transform:
    def __init__(self, affine=False, crop=False, gridmask=False, size=(128, 128),
                 normalize=True, train=True, threshold=40.,
                 sigma=-1., blur_ratio=0., noise_ratio=0., cutout_ratio=0.,
                 grid_distortion_ratio=0., elastic_distortion_ratio=0., random_brightness_ratio=0.,
                 piece_affine_ratio=0., ssr_ratio=0., rotate_ratio=0.):
        self.affine = affine
        self.crop = crop
        self.gridmask = gridmask
        self.size = size
        self.normalize = normalize
        self.train = train
        self.threshold = threshold / 255.
        self.sigma = sigma / 255.

        self.blur_ratio = blur_ratio
        self.noise_ratio = noise_ratio
        self.cutout_ratio = cutout_ratio
        self.grid_distortion_ratio = grid_distortion_ratio
        self.elastic_distortion_ratio = elastic_distortion_ratio
        self.random_brightness_ratio = random_brightness_ratio
        self.piece_affine_ratio = piece_affine_ratio
        self.ssr_ratio = ssr_ratio
        self.rotate_ratio = rotate_ratio
        #self.gmask_ratio = gmask_ratio

    def __call__(self, example):
        if self.train:
            x, y = example
        else:
            x = example
        # --- Augmentation ---
        if self.affine:
            x = affine_image(x)

        # --- Train/Test common preprocessing ---
        if self.crop:
            x = crop_char_image(x, threshold=self.threshold)
        if self.gridmask:
            x = apply_aug(GridMask(num_grid=3, p=0.2, mode=2), x)
        if self.size is not None:
            x = resize(x, size=self.size)
        if self.sigma > 0.:
            x = add_gaussian_noise(x, sigma=self.sigma)

        # albumentations...
        x = x.astype(np.float32)
        assert x.ndim == 2
        # 1. blur
        if _evaluate_ratio(self.blur_ratio):
            r = np.random.uniform()
            if r < 0.25:
                x = apply_aug(A.Blur(p=1.0), x)
            elif r < 0.5:
                x = apply_aug(A.MedianBlur(blur_limit=5, p=1.0), x)
            elif r < 0.75:
                x = apply_aug(A.GaussianBlur(p=1.0), x)
            else:
                x = apply_aug(A.MotionBlur(p=1.0), x)

        if _evaluate_ratio(self.noise_ratio):
            r = np.random.uniform()
            if r < 0.50:
                x = apply_aug(A.GaussNoise(var_limit=5. / 255., p=1.0), x)
            else:
                x = apply_aug(A.MultiplicativeNoise(p=1.0), x)

        if _evaluate_ratio(self.cutout_ratio):
            # A.Cutout(num_holes=2,  max_h_size=2, max_w_size=2, p=1.0)  # Deprecated...
            #x = apply_aug(A.CoarseDropout(max_holes=10, max_height=13, max_width=23, p=1.0), x)
            x = apply_aug(A.Cutout(num_holes=10, max_h_size=13, max_w_size=23, p=1), x)

        if _evaluate_ratio(self.grid_distortion_ratio):
            x = apply_aug(A.GridDistortion(p=1.0), x)

        if _evaluate_ratio(self.elastic_distortion_ratio):
            x = apply_aug(A.ElasticTransform(
                sigma=50, alpha=1, alpha_affine=10, p=1.0), x)

        if _evaluate_ratio(self.random_brightness_ratio):
            # A.RandomBrightness(p=1.0)  # Deprecated...
            # A.RandomContrast(p=1.0)    # Deprecated...
            x = apply_aug(A.RandomBrightnessContrast(p=1.0), x)

        if _evaluate_ratio(self.piece_affine_ratio):
            x = apply_aug(A.IAAPiecewiseAffine(p=1.0), x)

        if _evaluate_ratio(self.ssr_ratio):
            x = apply_aug(A.ShiftScaleRotate(
                shift_limit=0.0625,
                scale_limit=0.1,
                rotate_limit=30,
                p=1.0), x)
            
        if _evaluate_ratio(self.rotate_ratio):
            x = apply_aug(A.Rotate(limit=20), x)

        if self.normalize:
            x = (x.astype(np.float32) - 0.0692) / 0.2051
        if x.ndim == 2:
            x = x[None, :, :]
        x = x.astype(np.float32)
        if self.train:
            y = y.astype(np.int64)
            return x, y
        else:
            return x

<a id="model"></a> 
# pytorch model & define classifier

In [15]:
import torch


def residual_add(lhs, rhs):
    lhs_ch, rhs_ch = lhs.shape[1], rhs.shape[1]
    if lhs_ch < rhs_ch:
        out = lhs + rhs[:, :lhs_ch]
    elif lhs_ch > rhs_ch:
        out = torch.cat([lhs[:, :rhs_ch] + rhs, lhs[:, rhs_ch:]], dim=1)
    else:
        out = lhs + rhs
    return out


In [16]:
from typing import List

import torch
from torch import nn
from torch.nn.parameter import Parameter


class LazyLoadModule(nn.Module):
    """Lazy buffer/parameter loading using load_state_dict_pre_hook

    Define all buffer/parameter in `_lazy_buffer_keys`/`_lazy_parameter_keys` and
    save buffer with `register_buffer`/`register_parameter`
    method, which can be outside of __init__ method.
    Then this module can load any shape of Tensor during de-serializing.

    Note that default value of lazy buffer is torch.Tensor([]), while lazy parameter is None.
    """
    _lazy_buffer_keys: List[str] = []     # It needs to be override to register lazy buffer
    _lazy_parameter_keys: List[str] = []  # It needs to be override to register lazy parameter

    def __init__(self):
        super(LazyLoadModule, self).__init__()
        for k in self._lazy_buffer_keys:
            self.register_buffer(k, torch.tensor([]))
        for k in self._lazy_parameter_keys:
            self.register_parameter(k, None)
        self._register_load_state_dict_pre_hook(self._hook)

    def _hook(self, state_dict, prefix, local_metadata, strict, missing_keys,
             unexpected_keys, error_msgs):
        for key in self._lazy_buffer_keys:
            self.register_buffer(key, state_dict[prefix + key])

        for key in self._lazy_parameter_keys:
            self.register_parameter(key, Parameter(state_dict[prefix + key]))


In [17]:
import math
import torch
from torch.nn import init
from torch.nn.parameter import Parameter
import torch.nn.functional as F


class LazyLinear(LazyLoadModule):
    """Linear module with lazy input inference

    `in_features` can be `None`, and it is determined at the first time of forward step dynamically.
    """

    __constants__ = ['bias', 'in_features', 'out_features']
    _lazy_parameter_keys = ['weight']

    def __init__(self, in_features, out_features, bias=True):
        super(LazyLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)

        if in_features is not None:
            self.weight = Parameter(torch.Tensor(out_features, in_features))
            self.reset_parameters()

    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        if self.weight is None:
            self.in_features = input.shape[-1]
            self.weight = Parameter(torch.Tensor(self.out_features, self.in_features))
            self.reset_parameters()

            # Need to send lazy defined parameter to device...
            self.to(input.device)
        return F.linear(input, self.weight, self.bias)

    def extra_repr(self):
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )


In [18]:
from torch import nn
import torch.nn.functional as F


class LinearBlock(nn.Module):

    def __init__(self, in_features, out_features, bias=True,
                 use_bn=True, activation=F.relu, dropout_ratio=-1, residual=False,):
        super(LinearBlock, self).__init__()
        if in_features is None:
            self.linear = LazyLinear(in_features, out_features, bias=bias)
        else:
            self.linear = nn.Linear(in_features, out_features, bias=bias)
        if use_bn:
            self.bn = nn.BatchNorm1d(out_features)
        if dropout_ratio > 0.:
            self.dropout = nn.Dropout(p=dropout_ratio)
        else:
            self.dropout = None
        self.activation = activation
        self.use_bn = use_bn
        self.dropout_ratio = dropout_ratio
        self.residual = residual

    def __call__(self, x):
        h = self.linear(x)
        if self.use_bn:
            h = self.bn(h)
        if self.activation is not None:
            h = self.activation(h)
        if self.residual:
            h = residual_add(h, x)
        if self.dropout_ratio > 0:
            h = self.dropout(h)
        return h

In [19]:
import pretrainedmodels
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Sequential


class PretrainedCNN(nn.Module):
    def __init__(self, model_name='se_resnext101_32x4d',
                 in_channels=1, out_dim=10, use_bn=True,
                 pretrained='imagenet'):
        super(PretrainedCNN, self).__init__()
        self.conv0 = nn.Conv2d(
            in_channels, 3, kernel_size=3, stride=1, padding=1, bias=True)
        self.base_model = pretrainedmodels.__dict__[model_name](pretrained=pretrained)
        activation = F.leaky_relu
        self.do_pooling = True
        if self.do_pooling:
            inch = self.base_model.last_linear.in_features
        else:
            inch = None
        hdim = 512
        lin1 = LinearBlock(inch, hdim, use_bn=use_bn, activation=activation, residual=False)
        lin2 = LinearBlock(hdim, out_dim, use_bn=use_bn, activation=None, residual=False)
        self.lin_layers = Sequential(lin1, lin2)

    def forward(self, x):
        h = self.conv0(x)
        h = self.base_model.features(h)

        if self.do_pooling:
            h = torch.sum(h, dim=(-1, -2))
        else:
            # [128, 2048, 4, 4] when input is (128, 128)
            bs, ch, height, width = h.shape
            h = h.view(bs, ch*height*width)
        for layer in self.lin_layers:
            h = layer(h)
        return h

## Classifier

In [20]:
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm


def accuracy(y, t):
    pred_label = torch.argmax(y, dim=1)
    count = pred_label.shape[0]
    correct = (pred_label == t).sum().type(torch.float32)
    acc = correct / count
    return acc


class BengaliClassifier(nn.Module):
    def __init__(self, predictor, n_grapheme=168, n_vowel=11, n_consonant=7):
        super(BengaliClassifier, self).__init__()
        self.n_grapheme = n_grapheme
        self.n_vowel = n_vowel
        self.n_consonant = n_consonant
        self.n_total_class = self.n_grapheme + self.n_vowel + self.n_consonant
        self.predictor = predictor

        self.metrics_keys = [
            'loss', 'loss_grapheme', 'loss_vowel', 'loss_consonant',
            'acc_grapheme', 'acc_vowel', 'acc_consonant']

    def forward(self, x, y=None):
        pred = self.predictor(x)
        if isinstance(pred, tuple):
            assert len(pred) == 3
            preds = pred
        else:
            assert pred.shape[1] == self.n_total_class
            preds = torch.split(pred, [self.n_grapheme, self.n_vowel, self.n_consonant], dim=1)
        loss_grapheme = F.cross_entropy(preds[0], y[:, 0])
        loss_vowel = F.cross_entropy(preds[1], y[:, 1])
        loss_consonant = F.cross_entropy(preds[2], y[:, 2])
        loss = loss_grapheme + loss_vowel + loss_consonant
        metrics = {
            'loss': loss.item(),
            'loss_grapheme': loss_grapheme.item(),
            'loss_vowel': loss_vowel.item(),
            'loss_consonant': loss_consonant.item(),
            'acc_grapheme': accuracy(preds[0], y[:, 0]),
            'acc_vowel': accuracy(preds[1], y[:, 1]),
            'acc_consonant': accuracy(preds[2], y[:, 2]),
        }
        return loss, metrics, pred

    def calc(self, data_loader):
        device: torch.device = next(self.parameters()).device
        self.eval()
        output_list = []
        with torch.no_grad():
            for batch in tqdm(data_loader):
                # TODO: support general preprocessing.
                # If `data` is not `Data` instance, `to` method is not supported!
                batch = batch.to(device)
                pred = self.predictor(batch)
                output_list.append(pred)
        output = torch.cat(output_list, dim=0)
        preds = torch.split(output, [self.n_grapheme, self.n_vowel, self.n_consonant], dim=1)
        return preds

    def predict_proba(self, data_loader):
        preds = self.calc(data_loader)
        return [F.softmax(p, dim=1) for p in preds]

    def predict(self, data_loader):
        preds = self.calc(data_loader)
        pred_labels = [torch.argmax(p, dim=1) for p in preds]
        return pred_labels


<a id="train"></a>
# Training code

## prepare data

In [21]:
train_transform = Transform(size=(image_size, image_size), 
                            threshold=20., gridmask=False, cutout_ratio=1.0, rotate_ratio=1.0)

valid_transform = Transform(size=(image_size, image_size), threshold=20., gridmask=False)

In [22]:
%%time
msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2 ,random_state=SEED)

CPU times: user 6 µs, sys: 7 µs, total: 13 µs
Wall time: 14.8 µs


In [23]:
for tr, val in msss.split(train_images, train_labels):
    tr_idx = tr
    val_idx = val

In [24]:
#n_dataset = len(train_images)
#train_data_size = 200 if debug else int(n_dataset * 0.8)
#valid_data_size = 100 if debug else int(n_dataset - train_data_size)

#perm = np.random.RandomState(777).permutation(n_dataset)
#print('perm', perm)

train_dataset = BengaliAIDataset(
    train_images, train_labels, transform=train_transform,
    indices=tr_idx)

valid_dataset = BengaliAIDataset(
    train_images, train_labels, transform=valid_transform,
    indices=val_idx)
print('train_dataset', len(train_dataset), 'valid_dataset', len(valid_dataset))

train_dataset 160672 valid_dataset 40168


In [25]:
# --- Model ---
device = torch.device(device)
n_grapheme = 168
n_vowel = 11
n_consonant = 7
n_total = n_grapheme + n_vowel + n_consonant
print('n_total', n_total)
# Set pretrained='imagenet' to download imagenet pretrained model...
predictor = PretrainedCNN(in_channels=1, out_dim=n_total, model_name=model_name, pretrained=None)
print('predictor', type(predictor))

classifier = BengaliClassifier(predictor).to(device)

n_total 186
predictor <class '__main__.PretrainedCNN'>


## Ignite utility

pytorch-ignite utility class for training

In [26]:
import json
from logging import getLogger
import numpy

def save_json(filepath, params):
    with open(filepath, 'w') as f:
        json.dump(params, f, indent=4)

In [27]:
import os
from logging import getLogger
from time import perf_counter

import pandas as pd
import torch
# from chainer_chemistry.utils import save_json

from ignite.engine.engine import Engine, Events
from ignite.metrics import Average


class DictOutputTransform:
    def __init__(self, key, index=0):
        self.key = key
        self.index = index

    def __call__(self, x):
        if self.index >= 0:
            x = x[self.index]
        return x[self.key]


def create_trainer(classifier, optimizer, device):
    classifier.to(device)

    def update_fn(engine, batch):
        classifier.train()
        optimizer.zero_grad()
        # batch = [elem.to(device) for elem in batch]
        x, y = [elem.to(device) for elem in batch]
        loss, metrics, pred_y = classifier(x, y)
        loss.backward()
        optimizer.step()
        return metrics, pred_y, y
    trainer = Engine(update_fn)

    for key in classifier.metrics_keys:
        Average(output_transform=DictOutputTransform(key)).attach(trainer, key)
    return trainer


def create_evaluator(classifier, device):
    classifier.to(device)

    def update_fn(engine, batch):
        classifier.eval()
        with torch.no_grad():
            # batch = [elem.to(device) for elem in batch]
            x, y = [elem.to(device) for elem in batch]
            _, metrics, pred_y = classifier(x, y)
            return metrics, pred_y, y
    evaluator = Engine(update_fn)

    for key in classifier.metrics_keys:
        Average(output_transform=DictOutputTransform(key)).attach(evaluator, key)
    return evaluator


class LogReport:
    def __init__(self, evaluator=None, dirpath=None, logger=None):
        self.evaluator = evaluator
        self.dirpath = str(dirpath) if dirpath is not None else None
        self.logger = logger or getLogger(__name__)

        self.reported_dict = {}  # To handle additional parameter to monitor
        self.history = []
        self.start_time = perf_counter()

    def report(self, key, value):
        self.reported_dict[key] = value

    def __call__(self, engine):
        elapsed_time = perf_counter() - self.start_time
        elem = {'epoch': engine.state.epoch,
                'iteration': engine.state.iteration}
        elem.update({f'train/{key}': value
                     for key, value in engine.state.metrics.items()})
        if self.evaluator is not None:
            elem.update({f'valid/{key}': value
                         for key, value in self.evaluator.state.metrics.items()})
        elem.update(self.reported_dict)
        elem['elapsed_time'] = elapsed_time
        self.history.append(elem)
        if self.dirpath:
            save_json(os.path.join(self.dirpath, 'log.json'), self.history)
            self.get_dataframe().to_csv(os.path.join(self.dirpath, 'log.csv'), index=False)

        # --- print ---
        msg = ''
        for key, value in elem.items():
            if key in ['iteration']:
                # skip printing some parameters...
                continue
            elif isinstance(value, int):
                msg += f'{key} {value: >6d} '
            else:
                msg += f'{key} {value: 8f} '
#         self.logger.warning(msg)
        print(msg)

        # --- Reset ---
        self.reported_dict = {}

    def get_dataframe(self):
        df = pd.DataFrame(self.history)
        return df


class SpeedCheckHandler:
    def __init__(self, iteration_interval=10, logger=None):
        self.iteration_interval = iteration_interval
        self.logger = logger or getLogger(__name__)
        self.prev_time = perf_counter()

    def __call__(self, engine: Engine):
        if engine.state.iteration % self.iteration_interval == 0:
            cur_time = perf_counter()
            spd = self.iteration_interval / (cur_time - self.prev_time)
            self.logger.warning(f'{spd} iter/sec')
            # reset
            self.prev_time = cur_time

    def attach(self, engine: Engine):
        engine.add_event_handler(Events.ITERATION_COMPLETED, self)


class ModelSnapshotHandler:
    def __init__(self, model, filepath='model_{count:06}.pt',
                 interval=1, logger=None):
        self.model = model
        self.filepath: str = str(filepath)
        self.interval = interval
        self.logger = logger or getLogger(__name__)
        self.count = 0

    def __call__(self, engine: Engine):
        self.count += 1
        if self.count % self.interval == 0:
            filepath = self.filepath.format(count=self.count)
            torch.save(self.model.state_dict(), filepath)
            # self.logger.warning(f'save model to {filepath}...')


In [28]:
import warnings

import torch

from ignite.metrics.metric import Metric


class EpochMetric(Metric):
    """Class for metrics that should be computed on the entire output history of a model.
    Model's output and targets are restricted to be of shape `(batch_size, n_classes)`. Output
    datatype should be `float32`. Target datatype should be `long`.

    .. warning::

        Current implementation stores all input data (output and target) in as tensors before computing a metric.
        This can potentially lead to a memory error if the input data is larger than available RAM.


    - `update` must receive output of the form `(y_pred, y)`.

    If target shape is `(batch_size, n_classes)` and `n_classes > 1` than it should be binary: e.g. `[[0, 1, 0, 1], ]`.

    Args:
        compute_fn (callable): a callable with the signature (`torch.tensor`, `torch.tensor`) takes as the input
            `predictions` and `targets` and returns a scalar.
        output_transform (callable, optional): a callable that is used to transform the
            :class:`~ignite.engine.Engine`'s `process_function`'s output into the
            form expected by the metric. This can be useful if, for example, you have a multi-output model and
            you want to compute the metric with respect to one of the outputs.

    """

    def __init__(self, compute_fn, output_transform=lambda x: x):

        if not callable(compute_fn):
            raise TypeError("Argument compute_fn should be callable.")

        super(EpochMetric, self).__init__(output_transform=output_transform)
        self.compute_fn = compute_fn

    def reset(self):
        self._predictions = torch.tensor([], dtype=torch.float32)
        self._targets = torch.tensor([], dtype=torch.long)

    def update(self, output):
        y_pred, y = output
        self._predictions = torch.cat([self._predictions, y_pred], dim=0)
        self._targets = torch.cat([self._targets, y], dim=0)

        # Check once the signature and execution of compute_fn
        if self._predictions.shape == y_pred.shape:
            try:
                self.compute_fn(self._predictions, self._targets)
            except Exception as e:
                warnings.warn("Probably, there can be a problem with `compute_fn`:\n {}.".format(e),
                              RuntimeWarning)

    def compute(self):
        return self.compute_fn(self._predictions, self._targets)


In [29]:
import numpy as np
import sklearn.metrics
import torch


def macro_recall(pred_y, y, n_grapheme=168, n_vowel=11, n_consonant=7):
    pred_y = torch.split(pred_y, [n_grapheme, n_vowel, n_consonant], dim=1)
    pred_labels = [torch.argmax(py, dim=1).cpu().numpy() for py in pred_y]

    y = y.cpu().numpy()
    # pred_y = [p.cpu().numpy() for p in pred_y]

    recall_grapheme = sklearn.metrics.recall_score(pred_labels[0], y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(pred_labels[1], y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(pred_labels[2], y[:, 2], average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_score = np.average(scores, weights=[2, 1, 1])
    # print(f'recall: grapheme {recall_grapheme}, vowel {recall_vowel}, consonant {recall_consonant}, '
    #       f'total {final_score}, y {y.shape}')
    return final_score


def calc_macro_recall(solution, submission):
    # solution df, submission df
    scores = []
    for component in ['grapheme_root', 'consonant_diacritic', 'vowel_diacritic']:
        y_true_subset = solution[solution[component] == component]['target'].values
        y_pred_subset = submission[submission[component] == component]['target'].values
        scores.append(sklearn.metrics.recall_score(
            y_true_subset, y_pred_subset, average='macro'))
    final_score = np.average(scores, weights=[2, 1, 1])
    return final_score


In [30]:
import argparse
from distutils.util import strtobool
import os

import torch
from ignite.contrib.handlers import ProgressBar
from ignite.engine import Events
from numpy.random.mtrand import RandomState
from torch.utils.data.dataloader import DataLoader

# --- Training setting ---

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=8)


optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.7, patience=5, min_lr=1e-10)

trainer = create_trainer(classifier, optimizer, device)
def output_transform(output):
    metric, pred_y, y = output
    return pred_y.cpu(), y.cpu()
EpochMetric(
    compute_fn=macro_recall,
    output_transform=output_transform
).attach(trainer, 'recall')

pbar = ProgressBar()
pbar.attach(trainer, metric_names='all')

evaluator = create_evaluator(classifier, device)
EpochMetric(
    compute_fn=macro_recall,
    output_transform=output_transform
).attach(evaluator, 'recall')

def run_evaluator(engine):
    evaluator.run(valid_loader)

def schedule_lr(engine):
    metrics = evaluator.state.metrics
    #metrics = engine.state.metrics
    avg_mae = metrics['recall']

    # --- update lr ---
    lr = scheduler.optimizer.param_groups[0]['lr']
    scheduler.step(avg_mae)
    log_report.report('lr', lr)

def score_fn(engine):
    recall = evaluator.state.metrics['recall']
    return recall

trainer.add_event_handler(Events.EPOCH_COMPLETED, run_evaluator)
trainer.add_event_handler(Events.EPOCH_COMPLETED, schedule_lr)
log_report = LogReport(evaluator, outdir)
trainer.add_event_handler(Events.EPOCH_COMPLETED, log_report)
trainer.add_event_handler(
    Events.EPOCH_COMPLETED,
    ModelSnapshotHandler(predictor, filepath=outdir / '64_base_Cutout_v1_rotate_v2.pt'))

from ignite.handlers import Checkpoint, DiskSaver, global_step_from_engine

to_save = {'model': predictor}
handler = Checkpoint(to_save, DiskSaver('../bengali/model/64_base_Cutout_v1_rotate_v2/', require_empty=False), 
                     n_saved=1, filename_prefix='best_64_base_Cutout_v1_rotate_v2', score_function=score_fn,
                    score_name='recall', global_step_transform=global_step_from_engine(trainer))

evaluator.add_event_handler(Events.COMPLETED, handler)


In [31]:
save_json('64_base_Cutout_v1_rotate_v2.json', {'image_size':64, 'threshold':20,'arch':'pretrained','model_name':'se_resnext50_32x4d'})
trainer.run(train_loader, max_epochs=30)

epoch      1 train/loss  7.165123 train/loss_grapheme  4.456002 train/loss_vowel  1.377531 train/loss_consonant  1.331589 train/acc_grapheme  0.075275 train/acc_vowel  0.562583 train/acc_consonant  0.621535 train/recall  0.237206 valid/loss  4.344983 valid/loss_grapheme  3.052971 valid/loss_vowel  0.633133 valid/loss_consonant  0.658878 valid/acc_grapheme  0.255039 valid/acc_vowel  0.805021 valid/acc_consonant  0.853951 valid/recall  0.527162 lr  0.001000 elapsed_time  119.677340 


epoch      2 train/loss  4.891118 train/loss_grapheme  3.232455 train/loss_vowel  0.864334 train/loss_consonant  0.794329 train/acc_grapheme  0.237624 train/acc_vowel  0.721265 train/acc_consonant  0.781656 train/recall  0.486422 valid/loss  2.250212 valid/loss_grapheme  1.505020 valid/loss_vowel  0.349266 valid/loss_consonant  0.395926 valid/acc_grapheme  0.577054 valid/acc_vowel  0.899328 valid/acc_consonant  0.916862 valid/recall  0.755823 lr  0.001000 elapsed_time  243.062571 


epoch      3 train/loss  3.918113 train/loss_grapheme  2.599789 train/loss_vowel  0.695207 train/loss_consonant  0.623118 train/acc_grapheme  0.353621 train/acc_vowel  0.772213 train/acc_consonant  0.814140 train/recall  0.574989 valid/loss  1.686835 valid/loss_grapheme  1.058709 valid/loss_vowel  0.259206 valid/loss_consonant  0.368921 valid/acc_grapheme  0.694927 valid/acc_vowel  0.925540 valid/acc_consonant  0.907823 valid/recall  0.804287 lr  0.001000 elapsed_time  366.866566 


epoch      4 train/loss  3.429182 train/loss_grapheme  2.268525 train/loss_vowel  0.616707 train/loss_consonant  0.543950 train/acc_grapheme  0.424798 train/acc_vowel  0.795861 train/acc_consonant  0.828278 train/recall  0.619967 valid/loss  1.193877 valid/loss_grapheme  0.784612 valid/loss_vowel  0.192367 valid/loss_consonant  0.216898 valid/acc_grapheme  0.778749 valid/acc_vowel  0.946898 valid/acc_consonant  0.941465 valid/recall  0.844860 lr  0.001000 elapsed_time  489.742199 


epoch      5 train/loss  3.126261 train/loss_grapheme  2.075365 train/loss_vowel  0.560663 train/loss_consonant  0.490233 train/acc_grapheme  0.461895 train/acc_vowel  0.813625 train/acc_consonant  0.842191 train/recall  0.645399 valid/loss  0.916585 valid/loss_grapheme  0.611492 valid/loss_vowel  0.147341 valid/loss_consonant  0.157753 valid/acc_grapheme  0.825072 valid/acc_vowel  0.959292 valid/acc_consonant  0.959339 valid/recall  0.883065 lr  0.001000 elapsed_time  612.640927 


...

epoch     27 train/loss  1.496091 train/loss_grapheme  0.947522 train/loss_vowel  0.293402 train/loss_consonant  0.255168 train/acc_grapheme  0.736877 train/acc_vowel  0.900483 train/acc_consonant  0.911749 train/recall  0.825968 valid/loss  0.274092 valid/loss_grapheme  0.170272 valid/loss_vowel  0.051837 valid/loss_consonant  0.051983 valid/acc_grapheme  0.951039 valid/acc_vowel  0.986243 valid/acc_consonant  0.984287 valid/recall  0.964821 lr  0.001000 elapsed_time  3357.867648 


epoch     28 train/loss  1.468793 train/loss_grapheme  0.929774 train/loss_vowel  0.286606 train/loss_consonant  0.252414 train/acc_grapheme  0.742299 train/acc_vowel  0.902153 train/acc_consonant  0.911889 train/recall  0.829827 valid/loss  0.271670 valid/loss_grapheme  0.171895 valid/loss_vowel  0.049235 valid/loss_consonant  0.050541 valid/acc_grapheme  0.950044 valid/acc_vowel  0.987044 valid/acc_consonant  0.984108 valid/recall  0.962524 lr  0.001000 elapsed_time  3481.403148 


epoch     29 train/loss  1.447069 train/loss_grapheme  0.915243 train/loss_vowel  0.284229 train/loss_consonant  0.247597 train/acc_grapheme  0.745487 train/acc_vowel  0.903416 train/acc_consonant  0.914002 train/recall  0.832549 valid/loss  0.257835 valid/loss_grapheme  0.162796 valid/loss_vowel  0.046891 valid/loss_consonant  0.048148 valid/acc_grapheme  0.953102 valid/acc_vowel  0.987121 valid/acc_consonant  0.985258 valid/recall  0.964509 lr  0.001000 elapsed_time  3605.448753 


epoch     30 train/loss  1.438918 train/loss_grapheme  0.910313 train/loss_vowel  0.281090 train/loss_consonant  0.247515 train/acc_grapheme  0.746456 train/acc_vowel  0.904225 train/acc_consonant  0.913536 train/recall  0.834393 valid/loss  0.380334 valid/loss_grapheme  0.205563 valid/loss_vowel  0.101405 valid/loss_consonant  0.073366 valid/acc_grapheme  0.941731 valid/acc_vowel  0.969421 valid/acc_consonant  0.976229 valid/recall  0.950139 lr  0.001000 elapsed_time  3729.256970 


State:
	iteration: 18840
	epoch: 30
	epoch_length: 628
	max_epochs: 30
	output: <class 'tuple'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: 12

In [31]:
save_json('64_base_Cutout_v1_rotate_v1.json', {'image_size':64, 'threshold':20,'arch':'pretrained','model_name':'se_resnext50_32x4d'})
trainer.run(train_loader, max_epochs=30)

epoch      1 train/loss  7.293473 train/loss_grapheme  4.512447 train/loss_vowel  1.410219 train/loss_consonant  1.370807 train/acc_grapheme  0.067966 train/acc_vowel  0.549270 train/acc_consonant  0.610257 train/recall  0.221970 valid/loss  5.791408 valid/loss_grapheme  3.828449 valid/loss_vowel  1.047808 valid/loss_consonant  0.915151 valid/acc_grapheme  0.150177 valid/acc_vowel  0.675150 valid/acc_consonant  0.756758 valid/recall  0.454128 lr  0.001000 elapsed_time  123.212711 


epoch      2 train/loss  5.066083 train/loss_grapheme  3.347683 train/loss_vowel  0.896529 train/loss_consonant  0.821871 train/acc_grapheme  0.216910 train/acc_vowel  0.708505 train/acc_consonant  0.772712 train/recall  0.470391 valid/loss  2.463226 valid/loss_grapheme  1.620330 valid/loss_vowel  0.345545 valid/loss_consonant  0.497351 valid/acc_grapheme  0.549749 valid/acc_vowel  0.901376 valid/acc_consonant  0.897114 valid/recall  0.733390 lr  0.001000 elapsed_time  247.789580 


epoch      3 train/loss  4.061535 train/loss_grapheme  2.695245 train/loss_vowel  0.722138 train/loss_consonant  0.644152 train/acc_grapheme  0.333489 train/acc_vowel  0.760977 train/acc_consonant  0.806873 train/recall  0.557267 valid/loss  1.827126 valid/loss_grapheme  1.143692 valid/loss_vowel  0.321563 valid/loss_consonant  0.361870 valid/acc_grapheme  0.681497 valid/acc_vowel  0.902530 valid/acc_consonant  0.910172 valid/recall  0.788366 lr  0.001000 elapsed_time  373.222702 


epoch      4 train/loss  3.537764 train/loss_grapheme  2.340286 train/loss_vowel  0.638329 train/loss_consonant  0.559149 train/acc_grapheme  0.406873 train/acc_vowel  0.786816 train/acc_consonant  0.823158 train/recall  0.608069 valid/loss  1.307341 valid/loss_grapheme  0.890123 valid/loss_vowel  0.193954 valid/loss_consonant  0.223264 valid/acc_grapheme  0.742717 valid/acc_vowel  0.944721 valid/acc_consonant  0.942281 valid/recall  0.842390 lr  0.001000 elapsed_time  500.285458 


epoch      5 train/loss  3.206434 train/loss_grapheme  2.129281 train/loss_vowel  0.577596 train/loss_consonant  0.499557 train/acc_grapheme  0.449514 train/acc_vowel  0.806575 train/acc_consonant  0.838960 train/recall  0.635722 valid/loss  3.489675 valid/loss_grapheme  1.568886 valid/loss_vowel  1.471003 valid/loss_consonant  0.449786 valid/acc_grapheme  0.605050 valid/acc_vowel  0.613818 valid/acc_consonant  0.864729 valid/recall  0.737903 lr  0.001000 elapsed_time  625.140963 


...

epoch     27 train/loss  1.588475 train/loss_grapheme  1.008840 train/loss_vowel  0.310419 train/loss_consonant  0.269216 train/acc_grapheme  0.719918 train/acc_vowel  0.894576 train/acc_consonant  0.907350 train/recall  0.814596 valid/loss  0.380083 valid/loss_grapheme  0.220358 valid/loss_vowel  0.092585 valid/loss_consonant  0.067140 valid/acc_grapheme  0.937407 valid/acc_vowel  0.972163 valid/acc_consonant  0.979306 valid/recall  0.948838 lr  0.001000 elapsed_time  3371.045964 


epoch     28 train/loss  1.554826 train/loss_grapheme  0.988422 train/loss_vowel  0.301850 train/loss_consonant  0.264555 train/acc_grapheme  0.725789 train/acc_vowel  0.897615 train/acc_consonant  0.908652 train/recall  0.818769 valid/loss  0.289692 valid/loss_grapheme  0.183215 valid/loss_vowel  0.050920 valid/loss_consonant  0.055556 valid/acc_grapheme  0.947148 valid/acc_vowel  0.986181 valid/acc_consonant  0.983262 valid/recall  0.961172 lr  0.001000 elapsed_time  3494.293322 


epoch     29 train/loss  1.540121 train/loss_grapheme  0.976670 train/loss_vowel  0.301741 train/loss_consonant  0.261710 train/acc_grapheme  0.728563 train/acc_vowel  0.897072 train/acc_consonant  0.908509 train/recall  0.821867 valid/loss  0.281409 valid/loss_grapheme  0.180288 valid/loss_vowel  0.049229 valid/loss_consonant  0.051891 valid/acc_grapheme  0.948096 valid/acc_vowel  0.985681 valid/acc_consonant  0.985432 valid/recall  0.960220 lr  0.001000 elapsed_time  3619.245615 


epoch     30 train/loss  1.527313 train/loss_grapheme  0.971513 train/loss_vowel  0.295216 train/loss_consonant  0.260584 train/acc_grapheme  0.730150 train/acc_vowel  0.898787 train/acc_consonant  0.909129 train/recall  0.821811 valid/loss  0.285620 valid/loss_grapheme  0.181469 valid/loss_vowel  0.051833 valid/loss_consonant  0.052317 valid/acc_grapheme  0.948546 valid/acc_vowel  0.985170 valid/acc_consonant  0.983536 valid/recall  0.962163 lr  0.001000 elapsed_time  3744.699162 


State:
	iteration: 18840
	epoch: 30
	epoch_length: 628
	max_epochs: 30
	output: <class 'tuple'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: 12

In [31]:
save_json('64_base_Cutout_v1.json', {'image_size':64, 'threshold':20,'arch':'pretrained','model_name':'se_resnext50_32x4d'})
trainer.run(train_loader, max_epochs=30)

epoch      1 train/loss  6.918097 train/loss_grapheme  4.339515 train/loss_vowel  1.298491 train/loss_consonant  1.280091 train/acc_grapheme  0.090175 train/acc_vowel  0.592307 train/acc_consonant  0.646815 train/recall  0.257103 valid/loss  3.880045 valid/loss_grapheme  2.725903 valid/loss_vowel  0.553603 valid/loss_consonant  0.600538 valid/acc_grapheme  0.319274 valid/acc_vowel  0.842095 valid/acc_consonant  0.873888 valid/recall  0.594103 lr  0.001000 elapsed_time  122.987447 


epoch      2 train/loss  4.470985 train/loss_grapheme  2.956768 train/loss_vowel  0.775141 train/loss_consonant  0.739075 train/acc_grapheme  0.288484 train/acc_vowel  0.753319 train/acc_consonant  0.801305 train/recall  0.526328 valid/loss  2.359062 valid/loss_grapheme  1.538203 valid/loss_vowel  0.455269 valid/loss_consonant  0.365590 valid/acc_grapheme  0.578835 valid/acc_vowel  0.858869 valid/acc_consonant  0.913465 valid/recall  0.742294 lr  0.001000 elapsed_time  245.559123 


epoch      3 train/loss  3.557450 train/loss_grapheme  2.359398 train/loss_vowel  0.625566 train/loss_consonant  0.572487 train/acc_grapheme  0.405030 train/acc_vowel  0.796401 train/acc_consonant  0.831728 train/recall  0.612661 valid/loss  1.384595 valid/loss_grapheme  0.910390 valid/loss_vowel  0.226825 valid/loss_consonant  0.247380 valid/acc_grapheme  0.743175 valid/acc_vowel  0.935528 valid/acc_consonant  0.944492 valid/recall  0.836151 lr  0.001000 elapsed_time  369.028681 


epoch      4 train/loss  3.103094 train/loss_grapheme  2.059620 train/loss_vowel  0.547962 train/loss_consonant  0.495512 train/acc_grapheme  0.467640 train/acc_vowel  0.819386 train/acc_consonant  0.845804 train/recall  0.652043 valid/loss  1.063254 valid/loss_grapheme  0.694327 valid/loss_vowel  0.187199 valid/loss_consonant  0.181729 valid/acc_grapheme  0.800786 valid/acc_vowel  0.948187 valid/acc_consonant  0.954477 valid/recall  0.866600 lr  0.001000 elapsed_time  489.722700 


epoch      5 train/loss  2.846118 train/loss_grapheme  1.890774 train/loss_vowel  0.505482 train/loss_consonant  0.449862 train/acc_grapheme  0.505179 train/acc_vowel  0.831989 train/acc_consonant  0.856495 train/recall  0.678219 valid/loss  0.862204 valid/loss_grapheme  0.553161 valid/loss_vowel  0.137963 valid/loss_consonant  0.171080 valid/acc_grapheme  0.840017 valid/acc_vowel  0.962466 valid/acc_consonant  0.958986 valid/recall  0.893520 lr  0.001000 elapsed_time  611.505406 


...

epoch     27 train/loss  1.214845 train/loss_grapheme  0.758050 train/loss_vowel  0.245698 train/loss_consonant  0.211097 train/acc_grapheme  0.787709 train/acc_vowel  0.916370 train/acc_consonant  0.926041 train/recall  0.859214 valid/loss  0.258095 valid/loss_grapheme  0.162431 valid/loss_vowel  0.046646 valid/loss_consonant  0.049018 valid/acc_grapheme  0.952527 valid/acc_vowel  0.987390 valid/acc_consonant  0.984487 valid/recall  0.962949 lr  0.001000 elapsed_time  3337.724718 


epoch     28 train/loss  1.184432 train/loss_grapheme  0.739299 train/loss_vowel  0.237248 train/loss_consonant  0.207884 train/acc_grapheme  0.792806 train/acc_vowel  0.919692 train/acc_consonant  0.927224 train/recall  0.863117 valid/loss  0.235701 valid/loss_grapheme  0.147067 valid/loss_vowel  0.044147 valid/loss_consonant  0.044488 valid/acc_grapheme  0.957205 valid/acc_vowel  0.988244 valid/acc_consonant  0.986372 valid/recall  0.968351 lr  0.001000 elapsed_time  3462.138059 


epoch     29 train/loss  1.168572 train/loss_grapheme  0.728457 train/loss_vowel  0.234439 train/loss_consonant  0.205677 train/acc_grapheme  0.794673 train/acc_vowel  0.919765 train/acc_consonant  0.927992 train/recall  0.864098 valid/loss  0.283127 valid/loss_grapheme  0.182675 valid/loss_vowel  0.048130 valid/loss_consonant  0.052322 valid/acc_grapheme  0.947056 valid/acc_vowel  0.986753 valid/acc_consonant  0.983337 valid/recall  0.959658 lr  0.001000 elapsed_time  3586.188868 


epoch     30 train/loss  1.144471 train/loss_grapheme  0.713026 train/loss_vowel  0.230944 train/loss_consonant  0.200501 train/acc_grapheme  0.799016 train/acc_vowel  0.921922 train/acc_consonant  0.930980 train/recall  0.866955 valid/loss  0.236488 valid/loss_grapheme  0.149877 valid/loss_vowel  0.043260 valid/loss_consonant  0.043350 valid/acc_grapheme  0.956757 valid/acc_vowel  0.987810 valid/acc_consonant  0.986275 valid/recall  0.966948 lr  0.001000 elapsed_time  3711.584748 


State:
	iteration: 18840
	epoch: 30
	epoch_length: 628
	max_epochs: 30
	output: <class 'tuple'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: 12

In [31]:
save_json('64_base_CoarseDropout_v1.json', {'image_size':64, 'threshold':20,'arch':'pretrained','model_name':'se_resnext50_32x4d'})
trainer.run(train_loader, max_epochs=30)

epoch      1 train/loss  7.803825 train/loss_grapheme  4.770954 train/loss_vowel  1.616934 train/loss_consonant  1.415936 train/acc_grapheme  0.041543 train/acc_vowel  0.471339 train/acc_consonant  0.602777 train/recall  0.186982 valid/loss  5.926058 valid/loss_grapheme  4.080241 valid/loss_vowel  0.852660 valid/loss_consonant  0.993157 valid/acc_grapheme  0.097211 valid/acc_vowel  0.724992 valid/acc_consonant  0.741295 valid/recall  0.361428 lr  0.001000 elapsed_time  122.204459 


epoch      2 train/loss  6.414638 train/loss_grapheme  4.194132 train/loss_vowel  1.221037 train/loss_consonant  0.999468 train/acc_grapheme  0.084971 train/acc_vowel  0.587730 train/acc_consonant  0.717394 train/recall  0.334424 valid/loss  4.052573 valid/loss_grapheme  2.980132 valid/loss_vowel  0.549312 valid/loss_consonant  0.523129 valid/acc_grapheme  0.247766 valid/acc_vowel  0.819038 valid/acc_consonant  0.851388 valid/recall  0.547155 lr  0.001000 elapsed_time  244.002113 


epoch      3 train/loss  5.728765 train/loss_grapheme  3.804916 train/loss_vowel  1.079000 train/loss_consonant  0.844850 train/acc_grapheme  0.129717 train/acc_vowel  0.629266 train/acc_consonant  0.742951 train/recall  0.400224 valid/loss  4.420756 valid/loss_grapheme  2.708060 valid/loss_vowel  1.034343 valid/loss_consonant  0.678353 valid/acc_grapheme  0.325382 valid/acc_vowel  0.671751 valid/acc_consonant  0.779701 valid/recall  0.592301 lr  0.001000 elapsed_time  367.369016 


epoch      4 train/loss  5.248644 train/loss_grapheme  3.490509 train/loss_vowel  0.993063 train/loss_consonant  0.765072 train/acc_grapheme  0.176397 train/acc_vowel  0.656876 train/acc_consonant  0.757983 train/recall  0.456240 valid/loss  2.188198 valid/loss_grapheme  1.538526 valid/loss_vowel  0.329994 valid/loss_consonant  0.319677 valid/acc_grapheme  0.565377 valid/acc_vowel  0.902117 valid/acc_consonant  0.911961 valid/recall  0.760440 lr  0.001000 elapsed_time  490.545779 


epoch      5 train/loss  4.909870 train/loss_grapheme  3.256871 train/loss_vowel  0.940171 train/loss_consonant  0.712828 train/acc_grapheme  0.216042 train/acc_vowel  0.671926 train/acc_consonant  0.768055 train/recall  0.479279 valid/loss  1.789010 valid/loss_grapheme  1.246820 valid/loss_vowel  0.286938 valid/loss_consonant  0.255252 valid/acc_grapheme  0.651088 valid/acc_vowel  0.916434 valid/acc_consonant  0.926322 valid/recall  0.786259 lr  0.001000 elapsed_time  613.659064 


...

epoch     27 train/loss  2.557870 train/loss_grapheme  1.550849 train/loss_vowel  0.575627 train/loss_consonant  0.431395 train/acc_grapheme  0.579630 train/acc_vowel  0.796432 train/acc_consonant  0.848491 train/recall  0.709036 valid/loss  0.543465 valid/loss_grapheme  0.369789 valid/loss_vowel  0.085147 valid/loss_consonant  0.088529 valid/acc_grapheme  0.892036 valid/acc_vowel  0.974382 valid/acc_consonant  0.973136 valid/recall  0.927819 lr  0.001000 elapsed_time  3323.355878 


epoch     28 train/loss  2.501160 train/loss_grapheme  1.512216 train/loss_vowel  0.567566 train/loss_consonant  0.421378 train/acc_grapheme  0.591607 train/acc_vowel  0.799953 train/acc_consonant  0.851926 train/recall  0.716980 valid/loss  0.639386 valid/loss_grapheme  0.352446 valid/loss_vowel  0.175991 valid/loss_consonant  0.110949 valid/acc_grapheme  0.897456 valid/acc_vowel  0.939693 valid/acc_consonant  0.964649 valid/recall  0.917277 lr  0.001000 elapsed_time  3443.340005 


epoch     29 train/loss  2.356102 train/loss_grapheme  1.415540 train/loss_vowel  0.538524 train/loss_consonant  0.402037 train/acc_grapheme  0.614104 train/acc_vowel  0.809800 train/acc_consonant  0.857864 train/recall  0.731389 valid/loss  0.364998 valid/loss_grapheme  0.232654 valid/loss_vowel  0.065824 valid/loss_consonant  0.066520 valid/acc_grapheme  0.929826 valid/acc_vowel  0.981394 valid/acc_consonant  0.979876 valid/recall  0.945558 lr  0.000700 elapsed_time  3564.188750 


epoch     30 train/loss  2.293343 train/loss_grapheme  1.374453 train/loss_vowel  0.527388 train/loss_consonant  0.391503 train/acc_grapheme  0.624905 train/acc_vowel  0.813147 train/acc_consonant  0.861913 train/recall  0.740312 valid/loss  0.411115 valid/loss_grapheme  0.266453 valid/loss_vowel  0.071712 valid/loss_consonant  0.072950 valid/acc_grapheme  0.920779 valid/acc_vowel  0.979801 valid/acc_consonant  0.977938 valid/recall  0.943464 lr  0.000700 elapsed_time  3686.647771 


State:
	iteration: 18840
	epoch: 30
	epoch_length: 628
	max_epochs: 30
	output: <class 'tuple'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: 12

In [31]:
save_json('64_base.json', {'image_size':64, 'threshold':20,'arch':'pretrained','model_name':'se_resnext50_32x4d'})
trainer.run(train_loader, max_epochs=30)

epoch      1 train/loss  4.171247 train/loss_grapheme  2.606211 train/loss_vowel  0.715511 train/loss_consonant  0.849525 train/acc_grapheme  0.419847 train/acc_vowel  0.825901 train/acc_consonant  0.812347 train/recall  0.553260 valid/loss  1.979157 valid/loss_grapheme  1.227248 valid/loss_vowel  0.335061 valid/loss_consonant  0.416848 valid/acc_grapheme  0.692539 valid/acc_vowel  0.928070 valid/acc_consonant  0.935060 valid/recall  0.801647 lr  0.001000 elapsed_time  124.222851 


epoch      2 train/loss  1.323989 train/loss_grapheme  0.795397 train/loss_vowel  0.234398 train/loss_consonant  0.294193 train/acc_grapheme  0.792643 train/acc_vowel  0.951571 train/acc_consonant  0.951291 train/recall  0.854566 valid/loss  1.153544 valid/loss_grapheme  0.714464 valid/loss_vowel  0.184244 valid/loss_consonant  0.254836 valid/acc_grapheme  0.807913 valid/acc_vowel  0.956415 valid/acc_consonant  0.947197 valid/recall  0.875536 lr  0.001000 elapsed_time  246.806119 


epoch      3 train/loss  0.849020 train/loss_grapheme  0.528502 train/loss_vowel  0.147884 train/loss_consonant  0.172634 train/acc_grapheme  0.858755 train/acc_vowel  0.966460 train/acc_consonant  0.966401 train/recall  0.902738 valid/loss  0.857606 valid/loss_grapheme  0.527559 valid/loss_vowel  0.145069 valid/loss_consonant  0.184978 valid/acc_grapheme  0.857071 valid/acc_vowel  0.963788 valid/acc_consonant  0.958179 valid/recall  0.895327 lr  0.001000 elapsed_time  370.508769 


epoch      4 train/loss  0.639853 train/loss_grapheme  0.407201 train/loss_vowel  0.109346 train/loss_consonant  0.123306 train/acc_grapheme  0.888483 train/acc_vowel  0.974270 train/acc_consonant  0.973395 train/recall  0.922328 valid/loss  0.989925 valid/loss_grapheme  0.633759 valid/loss_vowel  0.157213 valid/loss_consonant  0.198953 valid/acc_grapheme  0.828766 valid/acc_vowel  0.957184 valid/acc_consonant  0.944727 valid/recall  0.904787 lr  0.001000 elapsed_time  495.201016 


epoch      5 train/loss  0.523077 train/loss_grapheme  0.337315 train/loss_vowel  0.089925 train/loss_consonant  0.095838 train/acc_grapheme  0.907141 train/acc_vowel  0.978155 train/acc_consonant  0.977610 train/recall  0.935990 valid/loss  0.696807 valid/loss_grapheme  0.465893 valid/loss_vowel  0.114271 valid/loss_consonant  0.116643 valid/acc_grapheme  0.872467 valid/acc_vowel  0.969620 valid/acc_consonant  0.970620 valid/recall  0.915160 lr  0.001000 elapsed_time  619.827661 


...

epoch     28 train/loss  0.054339 train/loss_grapheme  0.035179 train/loss_vowel  0.010038 train/loss_consonant  0.009122 train/acc_grapheme  0.989843 train/acc_vowel  0.997064 train/acc_consonant  0.997278 train/recall  0.992654 valid/loss  0.588444 valid/loss_grapheme  0.390590 valid/loss_vowel  0.095156 valid/loss_consonant  0.102699 valid/acc_grapheme  0.918429 valid/acc_vowel  0.980167 valid/acc_consonant  0.977910 valid/recall  0.943588 lr  0.001000 elapsed_time  3446.146600 


epoch     29 train/loss  0.049509 train/loss_grapheme  0.033363 train/loss_vowel  0.008113 train/loss_consonant  0.008033 train/acc_grapheme  0.990168 train/acc_vowel  0.997636 train/acc_consonant  0.997425 train/recall  0.993068 valid/loss  0.570967 valid/loss_grapheme  0.376788 valid/loss_vowel  0.093466 valid/loss_consonant  0.100712 valid/acc_grapheme  0.921841 valid/acc_vowel  0.980918 valid/acc_consonant  0.979105 valid/recall  0.944964 lr  0.001000 elapsed_time  3570.623273 


epoch     30 train/loss  0.057095 train/loss_grapheme  0.036917 train/loss_vowel  0.010689 train/loss_consonant  0.009490 train/acc_grapheme  0.989354 train/acc_vowel  0.996896 train/acc_consonant  0.997232 train/recall  0.992549 valid/loss  0.609704 valid/loss_grapheme  0.402507 valid/loss_vowel  0.096742 valid/loss_consonant  0.110455 valid/acc_grapheme  0.914509 valid/acc_vowel  0.978801 valid/acc_consonant  0.976144 valid/recall  0.941744 lr  0.001000 elapsed_time  3696.575355 


Exception ignored in: <function _releaseLock at 0x7fda2b4a5440>
Traceback (most recent call last):
  File "/home/yilgukseo/anaconda3/envs/pytorch/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Current run is terminating due to exception: DataLoader worker (pid(s) 5893) exited unexpectedly.
Engine run is terminating due to exception: DataLoader worker (pid(s) 5893) exited unexpectedly.


RuntimeError: DataLoader worker (pid(s) 5893) exited unexpectedly

In [31]:
save_json('msss_224_grid_mode_2.json', {'image_size':224, 'threshold':20,'arch':'pretrained','model_name':'se_resnext50_32x4d'})
trainer.run(train_loader, max_epochs=100)

epoch      1 train/loss  3.022022 train/loss_grapheme  1.974821 train/loss_vowel  0.512035 train/loss_consonant  0.535166 train/acc_grapheme  0.555985 train/acc_vowel  0.865890 train/acc_consonant  0.865666 train/recall  0.672736 valid/loss  0.904060 valid/loss_grapheme  0.575169 valid/loss_vowel  0.160677 valid/loss_consonant  0.168214 valid/acc_grapheme  0.850463 valid/acc_vowel  0.960082 valid/acc_consonant  0.959440 valid/recall  0.907979 lr  0.001000 elapsed_time  1117.822727 


epoch      2 train/loss  1.054836 train/loss_grapheme  0.711713 train/loss_vowel  0.175143 train/loss_consonant  0.167979 train/acc_grapheme  0.823595 train/acc_vowel  0.953604 train/acc_consonant  0.953847 train/recall  0.881407 valid/loss  0.566144 valid/loss_grapheme  0.342734 valid/loss_vowel  0.120870 valid/loss_consonant  0.102539 valid/acc_grapheme  0.908564 valid/acc_vowel  0.968894 valid/acc_consonant  0.972890 valid/recall  0.927318 lr  0.001000 elapsed_time  2259.994708 


epoch      3 train/loss  0.770951 train/loss_grapheme  0.521427 train/loss_vowel  0.126825 train/loss_consonant  0.122699 train/acc_grapheme  0.868323 train/acc_vowel  0.965676 train/acc_consonant  0.964425 train/recall  0.912850 valid/loss  0.460101 valid/loss_grapheme  0.285622 valid/loss_vowel  0.089533 valid/loss_consonant  0.084947 valid/acc_grapheme  0.924179 valid/acc_vowel  0.975522 valid/acc_consonant  0.976458 valid/recall  0.938076 lr  0.001000 elapsed_time  3398.423077 


epoch      4 train/loss  0.617681 train/loss_grapheme  0.416650 train/loss_vowel  0.102481 train/loss_consonant  0.098550 train/acc_grapheme  0.893599 train/acc_vowel  0.972216 train/acc_consonant  0.971003 train/recall  0.928458 valid/loss  0.379167 valid/loss_grapheme  0.237802 valid/loss_vowel  0.073195 valid/loss_consonant  0.068171 valid/acc_grapheme  0.937157 valid/acc_vowel  0.980359 valid/acc_consonant  0.981061 valid/recall  0.953530 lr  0.001000 elapsed_time  4540.553047 


epoch      5 train/loss  0.510320 train/loss_grapheme  0.342116 train/loss_vowel  0.085042 train/loss_consonant  0.083161 train/acc_grapheme  0.911402 train/acc_vowel  0.977088 train/acc_consonant  0.975321 train/recall  0.940937 valid/loss  0.338946 valid/loss_grapheme  0.210881 valid/loss_vowel  0.067580 valid/loss_consonant  0.060484 valid/acc_grapheme  0.943964 valid/acc_vowel  0.982315 valid/acc_consonant  0.982589 valid/recall  0.959850 lr  0.001000 elapsed_time  5670.801397 


...

epoch     59 train/loss  0.047013 train/loss_grapheme  0.028443 train/loss_vowel  0.008825 train/loss_consonant  0.009745 train/acc_grapheme  0.992047 train/acc_vowel  0.997156 train/acc_consonant  0.996963 train/recall  0.994365 valid/loss  0.166650 valid/loss_grapheme  0.085597 valid/loss_vowel  0.041952 valid/loss_consonant  0.039101 valid/acc_grapheme  0.979916 valid/acc_vowel  0.991446 valid/acc_consonant  0.991386 valid/recall  0.984454 lr  0.000700 elapsed_time  65532.365675 


epoch     60 train/loss  0.047557 train/loss_grapheme  0.029071 train/loss_vowel  0.008991 train/loss_consonant  0.009495 train/acc_grapheme  0.991842 train/acc_vowel  0.997362 train/acc_consonant  0.997113 train/recall  0.994762 valid/loss  0.167868 valid/loss_grapheme  0.088038 valid/loss_vowel  0.040912 valid/loss_consonant  0.038918 valid/acc_grapheme  0.978926 valid/acc_vowel  0.992257 valid/acc_consonant  0.990929 valid/recall  0.983002 lr  0.000700 elapsed_time  66623.630286 


epoch     61 train/loss  0.045062 train/loss_grapheme  0.026285 train/loss_vowel  0.008648 train/loss_consonant  0.010129 train/acc_grapheme  0.992508 train/acc_vowel  0.997536 train/acc_consonant  0.996864 train/recall  0.995010 valid/loss  0.169100 valid/loss_grapheme  0.090689 valid/loss_vowel  0.039560 valid/loss_consonant  0.038850 valid/acc_grapheme  0.978538 valid/acc_vowel  0.992367 valid/acc_consonant  0.991486 valid/recall  0.982839 lr  0.000700 elapsed_time  67728.881448 


Epoch [62/100]: [1573/2511]  63%|██████▎   , acc_consonant=0.997, acc_grapheme=0.993, acc_vowel=0.998, loss=0.0451, loss_consonant=0.0101, loss_grapheme=0.0263, loss_vowel=0.00865, recall=0.995 [10:39<06:36]Current run is terminating due to exception: .
Engine run is terminating due to exception: .


KeyboardInterrupt: 

In [32]:
save_json('msss_grid_mode_2.json', {'image_size':128, 'threshold':20,'arch':'pretrained','model_name':'se_resnext50_32x4d'})
trainer.run(train_loader, max_epochs=100)

epoch      1 train/loss  3.563295 train/loss_grapheme  2.276719 train/loss_vowel  0.608276 train/loss_consonant  0.678299 train/acc_grapheme  0.494657 train/acc_vowel  0.845206 train/acc_consonant  0.845436 train/recall  0.617920 valid/loss  1.372783 valid/loss_grapheme  0.869732 valid/loss_vowel  0.220886 valid/loss_consonant  0.282165 valid/acc_grapheme  0.774423 valid/acc_vowel  0.947912 valid/acc_consonant  0.947125 valid/recall  0.859391 lr  0.001000 elapsed_time  385.120160 


epoch      2 train/loss  1.265257 train/loss_grapheme  0.820875 train/loss_vowel  0.217984 train/loss_consonant  0.226399 train/acc_grapheme  0.793883 train/acc_vowel  0.943938 train/acc_consonant  0.945624 train/recall  0.861591 valid/loss  0.640386 valid/loss_grapheme  0.396924 valid/loss_vowel  0.115720 valid/loss_consonant  0.127741 valid/acc_grapheme  0.893037 valid/acc_vowel  0.971303 valid/acc_consonant  0.971198 valid/recall  0.922169 lr  0.001000 elapsed_time  771.924695 


epoch      3 train/loss  0.911043 train/loss_grapheme  0.606802 train/loss_vowel  0.151571 train/loss_consonant  0.152670 train/acc_grapheme  0.843632 train/acc_vowel  0.959127 train/acc_consonant  0.958704 train/recall  0.893775 valid/loss  0.579820 valid/loss_grapheme  0.369511 valid/loss_vowel  0.102825 valid/loss_consonant  0.107484 valid/acc_grapheme  0.900307 valid/acc_vowel  0.971806 valid/acc_consonant  0.971433 valid/recall  0.928116 lr  0.001000 elapsed_time  1159.936905 


epoch      4 train/loss  0.747602 train/loss_grapheme  0.503174 train/loss_vowel  0.123693 train/loss_consonant  0.120735 train/acc_grapheme  0.869271 train/acc_vowel  0.966044 train/acc_consonant  0.965677 train/recall  0.912464 valid/loss  0.461435 valid/loss_grapheme  0.287614 valid/loss_vowel  0.079672 valid/loss_consonant  0.094149 valid/acc_grapheme  0.921865 valid/acc_vowel  0.979550 valid/acc_consonant  0.973822 valid/recall  0.943225 lr  0.001000 elapsed_time  1546.186937 


epoch      5 train/loss  0.626498 train/loss_grapheme  0.423079 train/loss_vowel  0.102300 train/loss_consonant  0.101118 train/acc_grapheme  0.888914 train/acc_vowel  0.971879 train/acc_consonant  0.970504 train/recall  0.925809 valid/loss  0.394958 valid/loss_grapheme  0.255480 valid/loss_vowel  0.067645 valid/loss_consonant  0.071833 valid/acc_grapheme  0.929251 valid/acc_vowel  0.982823 valid/acc_consonant  0.979674 valid/recall  0.948019 lr  0.001000 elapsed_time  1930.590385 


...

epoch     97 train/loss  0.015357 train/loss_grapheme  0.009609 train/loss_vowel  0.002464 train/loss_consonant  0.003285 train/acc_grapheme  0.997282 train/acc_vowel  0.999247 train/acc_consonant  0.998992 train/recall  0.998297 valid/loss  0.176136 valid/loss_grapheme  0.095969 valid/loss_vowel  0.041153 valid/loss_consonant  0.039014 valid/acc_grapheme  0.978323 valid/acc_vowel  0.992991 valid/acc_consonant  0.991560 valid/recall  0.985192 lr  0.000082 elapsed_time  37473.931967 


epoch     98 train/loss  0.014264 train/loss_grapheme  0.008671 train/loss_vowel  0.002488 train/loss_consonant  0.003105 train/acc_grapheme  0.997462 train/acc_vowel  0.999297 train/acc_consonant  0.999098 train/recall  0.998313 valid/loss  0.173684 valid/loss_grapheme  0.094992 valid/loss_vowel  0.041394 valid/loss_consonant  0.037298 valid/acc_grapheme  0.978884 valid/acc_vowel  0.992668 valid/acc_consonant  0.991822 valid/recall  0.985533 lr  0.000082 elapsed_time  37863.477981 


epoch     99 train/loss  0.014380 train/loss_grapheme  0.009089 train/loss_vowel  0.002282 train/loss_consonant  0.003009 train/acc_grapheme  0.997462 train/acc_vowel  0.999316 train/acc_consonant  0.999073 train/recall  0.998320 valid/loss  0.178121 valid/loss_grapheme  0.097923 valid/loss_vowel  0.041911 valid/loss_consonant  0.038287 valid/acc_grapheme  0.978069 valid/acc_vowel  0.992748 valid/acc_consonant  0.991759 valid/recall  0.984927 lr  0.000082 elapsed_time  38247.930853 


epoch    100 train/loss  0.014493 train/loss_grapheme  0.009040 train/loss_vowel  0.002479 train/loss_consonant  0.002974 train/acc_grapheme  0.997524 train/acc_vowel  0.999322 train/acc_consonant  0.999123 train/recall  0.998551 valid/loss  0.178479 valid/loss_grapheme  0.097154 valid/loss_vowel  0.041869 valid/loss_consonant  0.039457 valid/acc_grapheme  0.978274 valid/acc_vowel  0.992649 valid/acc_consonant  0.991187 valid/recall  0.984967 lr  0.000082 elapsed_time  38632.950807 


State:
	iteration: 125600
	epoch: 100
	epoch_length: 1256
	max_epochs: 100
	output: <class 'tuple'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: 12

In [27]:
save_json('holdout_grid_mode_2.json', {'image_size':128, 'threshold':20,'arch':'pretrained','model_name':'se_resnext50_32x4d'})
trainer.run(train_loader, max_epochs=100)

epoch      1 train/loss  3.584928 train/loss_grapheme  2.288273 train/loss_vowel  0.610913 train/loss_consonant  0.685742 train/acc_grapheme  0.489768 train/acc_vowel  0.843414 train/acc_consonant  0.849777 train/recall  0.615093 valid/loss  1.548024 valid/loss_grapheme  1.001705 valid/loss_vowel  0.265492 valid/loss_consonant  0.280827 valid/acc_grapheme  0.745633 valid/acc_vowel  0.934928 valid/acc_consonant  0.938845 valid/recall  0.855216 lr  0.001000 elapsed_time  407.175129 


epoch      2 train/loss  1.260545 train/loss_grapheme  0.819732 train/loss_vowel  0.216081 train/loss_consonant  0.224733 train/acc_grapheme  0.793547 train/acc_vowel  0.944404 train/acc_consonant  0.946693 train/recall  0.860919 valid/loss  1.279086 valid/loss_grapheme  0.791014 valid/loss_vowel  0.246319 valid/loss_consonant  0.241754 valid/acc_grapheme  0.794203 valid/acc_vowel  0.928884 valid/acc_consonant  0.935598 valid/recall  0.870070 lr  0.001000 elapsed_time  809.413784 


epoch      3 train/loss  0.903315 train/loss_grapheme  0.600129 train/loss_vowel  0.151613 train/loss_consonant  0.151572 train/acc_grapheme  0.845952 train/acc_vowel  0.960042 train/acc_consonant  0.959065 train/recall  0.895812 valid/loss  0.840121 valid/loss_grapheme  0.553514 valid/loss_vowel  0.149072 valid/loss_consonant  0.137535 valid/acc_grapheme  0.852912 valid/acc_vowel  0.958490 valid/acc_consonant  0.961902 valid/recall  0.900351 lr  0.001000 elapsed_time  1216.151267 


epoch      4 train/loss  0.730620 train/loss_grapheme  0.487727 train/loss_vowel  0.122904 train/loss_consonant  0.119989 train/acc_grapheme  0.872997 train/acc_vowel  0.966287 train/acc_consonant  0.966100 train/recall  0.914604 valid/loss  0.696896 valid/loss_grapheme  0.463357 valid/loss_vowel  0.119141 valid/loss_consonant  0.114397 valid/acc_grapheme  0.875272 valid/acc_vowel  0.965119 valid/acc_consonant  0.966469 valid/recall  0.920578 lr  0.001000 elapsed_time  1621.663126 


epoch      5 train/loss  0.620513 train/loss_grapheme  0.412519 train/loss_vowel  0.105146 train/loss_consonant  0.102848 train/acc_grapheme  0.891521 train/acc_vowel  0.971064 train/acc_consonant  0.969714 train/recall  0.927972 valid/loss  0.577840 valid/loss_grapheme  0.382133 valid/loss_vowel  0.098083 valid/loss_consonant  0.097625 valid/acc_grapheme  0.896755 valid/acc_vowel  0.972074 valid/acc_consonant  0.971606 valid/recall  0.931747 lr  0.001000 elapsed_time  2030.018710 


...

epoch     97 train/loss  0.017113 train/loss_grapheme  0.010444 train/loss_vowel  0.003222 train/loss_consonant  0.003446 train/acc_grapheme  0.997095 train/acc_vowel  0.999036 train/acc_consonant  0.999030 train/recall  0.998106 valid/loss  0.212542 valid/loss_grapheme  0.119570 valid/loss_vowel  0.049034 valid/loss_consonant  0.043938 valid/acc_grapheme  0.970386 valid/acc_vowel  0.990036 valid/acc_consonant  0.989688 valid/recall  0.979898 lr  0.000118 elapsed_time  39383.698151 


epoch     98 train/loss  0.015918 train/loss_grapheme  0.009657 train/loss_vowel  0.002499 train/loss_consonant  0.003762 train/acc_grapheme  0.997294 train/acc_vowel  0.999254 train/acc_consonant  0.998800 train/recall  0.998337 valid/loss  0.208423 valid/loss_grapheme  0.117578 valid/loss_vowel  0.048018 valid/loss_consonant  0.042827 valid/acc_grapheme  0.971780 valid/acc_vowel  0.990670 valid/acc_consonant  0.989948 valid/recall  0.979846 lr  0.000082 elapsed_time  39788.317449 


epoch     99 train/loss  0.014872 train/loss_grapheme  0.009257 train/loss_vowel  0.002409 train/loss_consonant  0.003207 train/acc_grapheme  0.997226 train/acc_vowel  0.999303 train/acc_consonant  0.998961 train/recall  0.998410 valid/loss  0.210918 valid/loss_grapheme  0.118785 valid/loss_vowel  0.048549 valid/loss_consonant  0.043584 valid/acc_grapheme  0.971232 valid/acc_vowel  0.990589 valid/acc_consonant  0.989799 valid/recall  0.979697 lr  0.000082 elapsed_time  40196.554504 


epoch    100 train/loss  0.014623 train/loss_grapheme  0.008921 train/loss_vowel  0.002632 train/loss_consonant  0.003069 train/acc_grapheme  0.997468 train/acc_vowel  0.999160 train/acc_consonant  0.999123 train/recall  0.998336 valid/loss  0.209768 valid/loss_grapheme  0.116760 valid/loss_vowel  0.047879 valid/loss_consonant  0.045130 valid/acc_grapheme  0.971929 valid/acc_vowel  0.990341 valid/acc_consonant  0.989849 valid/recall  0.980860 lr  0.000082 elapsed_time  40608.674497 


State:
	iteration: 125600
	epoch: 100
	epoch_length: 1256
	max_epochs: 100
	output: <class 'tuple'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: 12

In [27]:
save_json('holdout_grid_mode_2.json', {'image_size':128, 'threshold':20,'arch':'pretrained','model_name':'se_resnext50_32x4d'})
trainer.run(train_loader, max_epochs=100)

epoch      1 train/loss  4.161853 train/loss_grapheme  2.620765 train/loss_vowel  0.777931 train/loss_consonant  0.763158 train/acc_grapheme  0.429949 train/acc_vowel  0.780479 train/acc_consonant  0.817171 train/recall  0.582633 valid/loss  2.486741 valid/loss_grapheme  1.595854 valid/loss_vowel  0.490085 valid/loss_consonant  0.400802 valid/acc_grapheme  0.616832 valid/acc_vowel  0.861142 valid/acc_consonant  0.897073 valid/recall  0.816198 lr  0.001000 elapsed_time  409.927902 


epoch      2 train/loss  1.940940 train/loss_grapheme  1.228146 train/loss_vowel  0.390169 train/loss_consonant  0.322624 train/acc_grapheme  0.713880 train/acc_vowel  0.882489 train/acc_consonant  0.913092 train/recall  0.822117 valid/loss  1.658773 valid/loss_grapheme  1.024045 valid/loss_vowel  0.358749 valid/loss_consonant  0.275979 valid/acc_grapheme  0.752865 valid/acc_vowel  0.887232 valid/acc_consonant  0.920727 valid/recall  0.854488 lr  0.001000 elapsed_time  833.808674 


epoch      3 train/loss  1.580412 train/loss_grapheme  1.008476 train/loss_vowel  0.319293 train/loss_consonant  0.252643 train/acc_grapheme  0.759523 train/acc_vowel  0.900633 train/acc_consonant  0.924506 train/recall  0.849164 valid/loss  1.475820 valid/loss_grapheme  0.941976 valid/loss_vowel  0.302591 valid/loss_consonant  0.231253 valid/acc_grapheme  0.764963 valid/acc_vowel  0.903012 valid/acc_consonant  0.928172 valid/recall  0.860847 lr  0.001000 elapsed_time  1268.058388 


epoch      4 train/loss  1.396083 train/loss_grapheme  0.895761 train/loss_vowel  0.282771 train/loss_consonant  0.217550 train/acc_grapheme  0.783402 train/acc_vowel  0.910654 train/acc_consonant  0.932468 train/recall  0.864683 valid/loss  1.414842 valid/loss_grapheme  0.873993 valid/loss_vowel  0.298510 valid/loss_consonant  0.242339 valid/acc_grapheme  0.786418 valid/acc_vowel  0.904095 valid/acc_consonant  0.923888 valid/recall  0.866085 lr  0.001000 elapsed_time  1688.320301 


epoch      5 train/loss  1.283603 train/loss_grapheme  0.823932 train/loss_vowel  0.258531 train/loss_consonant  0.201140 train/acc_grapheme  0.798349 train/acc_vowel  0.917751 train/acc_consonant  0.936181 train/recall  0.872666 valid/loss  1.248667 valid/loss_grapheme  0.786627 valid/loss_vowel  0.257177 valid/loss_consonant  0.204863 valid/acc_grapheme  0.802793 valid/acc_vowel  0.917354 valid/acc_consonant  0.933410 valid/recall  0.871521 lr  0.001000 elapsed_time  2099.605278 


...

epoch     18 train/loss  0.771554 train/loss_grapheme  0.480621 train/loss_vowel  0.166651 train/loss_consonant  0.124282 train/acc_grapheme  0.877003 train/acc_vowel  0.944417 train/acc_consonant  0.958014 train/recall  0.920864 valid/loss  1.012833 valid/loss_grapheme  0.607046 valid/loss_vowel  0.229254 valid/loss_consonant  0.176532 valid/acc_grapheme  0.846724 valid/acc_vowel  0.931221 valid/acc_consonant  0.945626 valid/recall  0.895426 lr  0.001000 elapsed_time  7418.949939 


epoch     19 train/loss  0.761347 train/loss_grapheme  0.475576 train/loss_vowel  0.162471 train/loss_consonant  0.123300 train/acc_grapheme  0.878900 train/acc_vowel  0.945225 train/acc_consonant  0.958866 train/recall  0.922922 valid/loss  0.913839 valid/loss_grapheme  0.557786 valid/loss_vowel  0.200783 valid/loss_consonant  0.155270 valid/acc_grapheme  0.854929 valid/acc_vowel  0.934530 valid/acc_consonant  0.950185 valid/recall  0.902548 lr  0.001000 elapsed_time  7829.596399 


epoch     20 train/loss  0.743052 train/loss_grapheme  0.462707 train/loss_vowel  0.158681 train/loss_consonant  0.121665 train/acc_grapheme  0.880530 train/acc_vowel  0.946494 train/acc_consonant  0.958897 train/recall  0.923288 valid/loss  0.891981 valid/loss_grapheme  0.543825 valid/loss_vowel  0.195133 valid/loss_consonant  0.153023 valid/acc_grapheme  0.855331 valid/acc_vowel  0.937969 valid/acc_consonant  0.951515 valid/recall  0.905822 lr  0.001000 elapsed_time  8234.409381 


Epoch [21/100]: [102/1256]   8%|▊         , acc_consonant=0.959, acc_grapheme=0.881, acc_vowel=0.946, loss=0.743, loss_consonant=0.122, loss_grapheme=0.463, loss_vowel=0.159, recall=0.923 [00:30<05:37]Current run is terminating due to exception: .
Engine run is terminating due to exception: .


KeyboardInterrupt: 

In [27]:
save_json('holdout_grid_mode_1.json', {'image_size':128, 'threshold':20,'arch':'pretrained','model_name':'se_resnext50_32x4d'})
trainer.run(train_loader, max_epochs=150)

epoch      1 train/loss  3.324080 train/loss_grapheme  2.099903 train/loss_vowel  0.587706 train/loss_consonant  0.636471 train/acc_grapheme  0.534304 train/acc_vowel  0.855494 train/acc_consonant  0.864158 train/recall  0.641159 valid/loss  1.633704 valid/loss_grapheme  1.070386 valid/loss_vowel  0.267908 valid/loss_consonant  0.295411 valid/acc_grapheme  0.722214 valid/acc_vowel  0.933626 valid/acc_consonant  0.932912 valid/recall  0.856382 lr  0.001000 elapsed_time  406.947959 


epoch      2 train/loss  1.060010 train/loss_grapheme  0.675211 train/loss_vowel  0.188300 train/loss_consonant  0.196498 train/acc_grapheme  0.829649 train/acc_vowel  0.954836 train/acc_consonant  0.956378 train/recall  0.883572 valid/loss  0.965976 valid/loss_grapheme  0.579340 valid/loss_vowel  0.184351 valid/loss_consonant  0.202284 valid/acc_grapheme  0.846295 valid/acc_vowel  0.951791 valid/acc_consonant  0.951596 valid/recall  0.888169 lr  0.001000 elapsed_time  817.111612 


epoch      3 train/loss  0.751487 train/loss_grapheme  0.491028 train/loss_vowel  0.131922 train/loss_consonant  0.128537 train/acc_grapheme  0.873625 train/acc_vowel  0.966044 train/acc_consonant  0.966791 train/recall  0.913083 valid/loss  0.715298 valid/loss_grapheme  0.468672 valid/loss_vowel  0.116274 valid/loss_consonant  0.130351 valid/acc_grapheme  0.874694 valid/acc_vowel  0.967992 valid/acc_consonant  0.965753 valid/recall  0.917099 lr  0.001000 elapsed_time  1226.521368 


epoch      4 train/loss  0.602182 train/loss_grapheme  0.396911 train/loss_vowel  0.104357 train/loss_consonant  0.100915 train/acc_grapheme  0.895831 train/acc_vowel  0.972619 train/acc_consonant  0.972793 train/recall  0.929519 valid/loss  0.725378 valid/loss_grapheme  0.454996 valid/loss_vowel  0.136821 valid/loss_consonant  0.133561 valid/acc_grapheme  0.878257 valid/acc_vowel  0.960790 valid/acc_consonant  0.961791 valid/recall  0.912926 lr  0.001000 elapsed_time  1636.793607 


epoch      5 train/loss  0.506383 train/loss_grapheme  0.331711 train/loss_vowel  0.088145 train/loss_consonant  0.086528 train/acc_grapheme  0.912713 train/acc_vowel  0.976749 train/acc_consonant  0.975629 train/recall  0.941138 valid/loss  0.519267 valid/loss_grapheme  0.338684 valid/loss_vowel  0.095289 valid/loss_consonant  0.085293 valid/acc_grapheme  0.908275 valid/acc_vowel  0.973217 valid/acc_consonant  0.975835 valid/recall  0.943264 lr  0.001000 elapsed_time  2047.036381 


...

epoch     67 train/loss  0.045674 train/loss_grapheme  0.028760 train/loss_vowel  0.008413 train/loss_consonant  0.008501 train/acc_grapheme  0.991746 train/acc_vowel  0.997506 train/acc_consonant  0.997437 train/recall  0.994852 valid/loss  0.289883 valid/loss_grapheme  0.168471 valid/loss_vowel  0.063401 valid/loss_consonant  0.058011 valid/acc_grapheme  0.959010 valid/acc_vowel  0.986938 valid/acc_consonant  0.985539 valid/recall  0.969534 lr  0.001000 elapsed_time  27298.464433 


epoch     68 train/loss  0.049271 train/loss_grapheme  0.030071 train/loss_vowel  0.009627 train/loss_consonant  0.009572 train/acc_grapheme  0.991547 train/acc_vowel  0.997195 train/acc_consonant  0.997126 train/recall  0.994500 valid/loss  0.293861 valid/loss_grapheme  0.166578 valid/loss_vowel  0.065597 valid/loss_consonant  0.061686 valid/acc_grapheme  0.959060 valid/acc_vowel  0.985962 valid/acc_consonant  0.985259 valid/recall  0.971326 lr  0.001000 elapsed_time  27702.777263 


epoch     69 train/loss  0.047162 train/loss_grapheme  0.029483 train/loss_vowel  0.008240 train/loss_consonant  0.009439 train/acc_grapheme  0.991702 train/acc_vowel  0.997475 train/acc_consonant  0.997269 train/recall  0.994533 valid/loss  0.292191 valid/loss_grapheme  0.172508 valid/loss_vowel  0.063890 valid/loss_consonant  0.055794 valid/acc_grapheme  0.957940 valid/acc_vowel  0.985992 valid/acc_consonant  0.986440 valid/recall  0.971084 lr  0.001000 elapsed_time  28110.777266 


Epoch [70/150]: [5/1256]   0%|          , acc_consonant=0.997, acc_grapheme=0.992, acc_vowel=0.997, loss=0.0472, loss_consonant=0.00944, loss_grapheme=0.0295, loss_vowel=0.00824, recall=0.995 [00:01<06:08]Current run is terminating due to exception: .
Engine run is terminating due to exception: .


KeyboardInterrupt: 

In [37]:
train_history = log_report.get_dataframe()
train_history.to_csv(outdir / 'holdout_grid_mode_1_log.csv', index=False)

train_history

,epoch,iteration,train/loss,train/loss_grapheme,train/loss_vowel,train/loss_consonant,train/acc_grapheme,train/acc_vowel,train/acc_consonant,train/recall,valid/loss,valid/loss_grapheme,valid/loss_vowel,valid/loss_consonant,valid/acc_grapheme,valid/acc_vowel,valid/acc_consonant,valid/recall,lr,elapsed_time
0,1,5021,2.799690,1.937696,0.438678,0.423316,0.548054,0.884448,0.889875,0.677631,3.301519,2.287018,0.614974,0.399527,0.503583,0.807200,0.870347,0.712342,0.001,1763.932305
1,2,10042,0.934554,0.648827,0.150129,0.135598,0.842381,0.961711,0.962339,0.891414,2.819982,1.833402,0.607212,0.379368,0.583624,0.804240,0.867586,0.746693,0.001,3496.321592
2,3,15063,0.629416,0.429597,0.104538,0.095282,0.893640,0.973630,0.973125,0.926838,3.224983,1.949107,0.850719,0.425157,0.583076,0.745621,0.847830,0.736990,0.001,5133.226001
3,4,20084,0.487317,0.329513,0.081662,0.076143,0.916482,0.979262,0.978546,0.941050,2.919880,1.771501,0.829959,0.318420,0.633410,0.783912,0.891720,0.790351,0.001,6701.143154
4,5,25105,0.395184,0.265151,0.067152,0.062880,0.930722,0.983052,0.982418,0.951739,2.278028,1.476290,0.480015,0.321723,0.664485,0.844397,0.885997,0.775654,0.001,8267.086915
5,6,30126,0.328522,0.219461,0.056008,0.053053,0.942131,0.985922,0.984839,0.959843,2.651598,1.541781,0.742678,0.367139,0.666302,0.784659,0.873930,0.788721,0.001,9834.383235
6,7,35147,0.278753,0.185525,0.046954,0.046274,0.950352,0.987994,0.986743,0.964962,2.628079,1.729377,0.532308,0.366393,0.642292,0.831633,0.880374,0.778568,0.001,11401.532617
7,8,40168,0.237421,0.158077,0.040693,0.038651,0.957286,0.989731,0.989183,0.970173,2.496537,1.623953,0.553325,0.319259,0.662769,0.837082,0.891023,0.779778,0.001,12970.003869
8,9,45189,0.206228,0.136699,0.035583,0.033946,0.963018,0.990353,0.990154,0.973625,2.469742,1.506492,0.607400,0.355850,0.678045,0.825662,0.885052,0.797764,0.001,14536.608497
9,10,50210,0.182064,0.120264,0.031452,0.030348,0.967250,0.991616,0.991019,0.977044,2.964096,1.825867,0.657815,0.480414,0.637614,0.822353,0.849249,0.751698,0.001,16101.024028


<a id="pred"></a>
# Prediction

Please refer **[Bengali: SEResNeXt prediction with pytorch](https://www.kaggle.com/corochann/bengali-seresnext-prediction-with-pytorch)** for the prediction with trained model and submission for this competition!!!

<a id="ref"></a>
# Reference and further reading

#### Kernel

**[Bangali.AI super fast data loading with feather](https://www.kaggle.com/corochann/bangali-ai-super-fast-data-loading-with-feather)**<br>
Simple example of how use feather format data to load data faster.

**[Bengali: albumentations data augmentation tutorial](https://www.kaggle.com/corochann/bengali-albumentations-data-augmentation-tutorial)**<br>
Tutorial for Data augmentations with albumentations library.

**[Bengali: SEResNeXt prediction with pytorch](https://www.kaggle.com/corochann/bengali-seresnext-prediction-with-pytorch)**<br>
**Prediction code of this kernel's trained model, please check this too!**

**[Deep learning - CNN with Chainer: LB 0.99700](https://www.kaggle.com/corochann/deep-learning-cnn-with-chainer-lb-0-99700)**<br>
Data augmentation idea is based on this kernel, which achieves quite high accuracy on MNIST task.

#### Dataset
**[bengaliai-cv19-feather](https://www.kaggle.com/corochann/bengaliaicv19feather)**<br>
Feather format dataset

**[bengaliaicv19_seresnext101_32x4d](https://www.kaggle.com/corochann/bengaliaicv19-seresnext101-32x4d)**<br>
**Trained model weight with this kernel(v1)**

**[bengaliaicv19_trainedmodels](https://www.kaggle.com/corochann/bengaliaicv19-trainedmodels)**<br>
**Trained model weight with this kernel(v2~)**

#### Library
**https://github.com/pytorch/ignite**

Used for training code abstraction. The advantage of abstracting the code is that we can re-use implemented handler class for other training, other competition.<br>
You don't need to write code for saving models, logging training loss/metric, show progressbar etc.

**https://github.com/Cadene/pretrained-models.pytorch**

Many pretrained models are supported by this library, and we can switch to use them easily.
Other model may perform better in this competition.

**https://github.com/albumentations-team/albumentations**

fast image augmentation library and easy to use wrapper around other libraries https://arxiv.org/abs/1809.06839<br>
I could not show all the methods, you can find more methods in the library, check yourself!

<h3 style="color:red">If this kernel helps you, please upvote to keep me motivated :)<br>Thanks!</h3>